**ECSE415_Final_Project**

Shiyuan Qiao: 260967649

Zhiheng Zhou: 260955157

In [ ]:
!git clone https://github.com/mohamedamine99/Object-tracking-and-counting-using-YOLOV8

Cloning into 'Object-tracking-and-counting-using-YOLOV8'...
remote: Enumerating objects: 249, done.
remote: Counting objects: 100% (51/51), done.
remote: Compressing objects: 100% (47/47), done.
remote: Total 249 (delta 27), reused 15 (delta 4), pack-reused 198
Receiving objects: 100% (249/249), 489.13 MiB | 14.45 MiB/s, done.
Resolving deltas: 100% (77/77), done.
Updating files: 100% (83/83), done.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import shutil
import os

src_path = "/content/Object-tracking-and-counting-using-YOLOV8/sort.py"
dest_path = "/content"

# copy file from source to destination
shutil.copy(src_path, dest_path)

src_path = "/content/Object-tracking-and-counting-using-YOLOV8/requirements.txt"
shutil.copy(src_path, dest_path)

src_path = "/content/Object-tracking-and-counting-using-YOLOV8/yolo_detect_and_count.py"
shutil.copy(src_path, dest_path)


'/content/yolo_detect_and_count.py'

In [ ]:
%cd /content/Object-tracking-and-counting-using-YOLOV8
!pip install -r requirements.txt

/content/Object-tracking-and-counting-using-YOLOV8
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 37.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for filterpy: filename=filterpy-1.4.5-py3-none-any.whl size=110458 sha256=37883b2c87ea7b22f1d00ff2a168cd3aac06570b22034ee25e93e5cebad6a650
  Stored in directory: /root/.cache/pip/wheels/0f/0c/ea/218f266af4ad626897562199fbbcba521b8497303200186102
  Created wheel for lap: filename=lap-0.4.0-cp310-cp310-linux_x86_64.whl size=1628967 sha256=c17d9a7c27d087b853e1dd79cc2c915b202ec1b9a1eed88859bfb30165145cfb
  Stored in directory: /root/.cache/pip/wheels/00/42/2e/9dfe19270eea279d79e84767ff0d7b8082c3bf776cad00e83d
Successfully built filterpy lap


In [ ]:
import numpy as np
import random
import cv2
# import sort
import time

In [ ]:
%pip install ultralytics
import ultralytics
ultralytics.checks()
from ultralytics import YOLO

Ultralytics YOLOv8.0.222 🚀 Python-3.10.12 torch-2.1.0+cu118 CPU (Intel Xeon 2.20GHz)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 27.9/107.7 GB disk)


In [ ]:
!pwd

/content/Object-tracking-and-counting-using-YOLOV8


In [ ]:
"""
    SORT: A Simple, Online and Realtime Tracker
    Copyright (C) 2016-2020 Alex Bewley alex@bewley.ai

    This program is free software: you can redistribute it and/or modify
    it under the terms of the GNU General Public License as published by
    the Free Software Foundation, either version 3 of the License, or
    (at your option) any later version.

    This program is distributed in the hope that it will be useful,
    but WITHOUT ANY WARRANTY; without even the implied warranty of
    MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
    GNU General Public License for more details.

    You should have received a copy of the GNU General Public License
    along with this program.  If not, see <http://www.gnu.org/licenses/>.
"""
from __future__ import print_function
#import matplotlib
#matplotlib.use('TkAgg')
import os
import numpy as np
#import matplotlib.pyplot as plt
#import matplotlib.patches as patches
from skimage import io

import glob
import time
import argparse
from filterpy.kalman import KalmanFilter

np.random.seed(0)


def linear_assignment(cost_matrix):
  try:
    import lap
    _, x, y = lap.lapjv(cost_matrix, extend_cost=True)
    return np.array([[y[i],i] for i in x if i >= 0]) #
  except ImportError:
    from scipy.optimize import linear_sum_assignment
    x, y = linear_sum_assignment(cost_matrix)
    return np.array(list(zip(x, y)))


def iou_batch(bb_test, bb_gt):
  """
  From SORT: Computes IOU between two bboxes in the form [x1,y1,x2,y2]
  """
  bb_gt = np.expand_dims(bb_gt, 0)
  bb_test = np.expand_dims(bb_test, 1)

  xx1 = np.maximum(bb_test[..., 0], bb_gt[..., 0])
  yy1 = np.maximum(bb_test[..., 1], bb_gt[..., 1])
  xx2 = np.minimum(bb_test[..., 2], bb_gt[..., 2])
  yy2 = np.minimum(bb_test[..., 3], bb_gt[..., 3])
  w = np.maximum(0., xx2 - xx1)
  h = np.maximum(0., yy2 - yy1)
  wh = w * h
  o = wh / ((bb_test[..., 2] - bb_test[..., 0]) * (bb_test[..., 3] - bb_test[..., 1])
    + (bb_gt[..., 2] - bb_gt[..., 0]) * (bb_gt[..., 3] - bb_gt[..., 1]) - wh)
  return(o)


def convert_bbox_to_z(bbox):
  """
  Takes a bounding box in the form [x1,y1,x2,y2] and returns z in the form
    [x,y,s,r] where x,y is the centre of the box and s is the scale/area and r is
    the aspect ratio
  """
  w = bbox[2] - bbox[0]
  h = bbox[3] - bbox[1]
  x = bbox[0] + w/2.
  y = bbox[1] + h/2.
  s = w * h    #scale is just area
  r = w / float(h)
  return np.array([x, y, s, r]).reshape((4, 1))


def convert_x_to_bbox(x,score=None):
  """
  Takes a bounding box in the centre form [x,y,s,r] and returns it in the form
    [x1,y1,x2,y2] where x1,y1 is the top left and x2,y2 is the bottom right
  """
  w = np.sqrt(x[2] * x[3])
  h = x[2] / w
  if(score==None):
    return np.array([x[0]-w/2.,x[1]-h/2.,x[0]+w/2.,x[1]+h/2.]).reshape((1,4))
  else:
    return np.array([x[0]-w/2.,x[1]-h/2.,x[0]+w/2.,x[1]+h/2.,score]).reshape((1,5))


class KalmanBoxTracker(object):
  """
  This class represents the internal state of individual tracked objects observed as bbox.
  """
  count = 0
  def __init__(self,bbox):
    """
    Initialises a tracker using initial bounding box.
    """
    #define constant velocity model
    self.kf = KalmanFilter(dim_x=7, dim_z=4)
    self.kf.F = np.array([[1,0,0,0,1,0,0],[0,1,0,0,0,1,0],[0,0,1,0,0,0,1],[0,0,0,1,0,0,0],  [0,0,0,0,1,0,0],[0,0,0,0,0,1,0],[0,0,0,0,0,0,1]])
    self.kf.H = np.array([[1,0,0,0,0,0,0],[0,1,0,0,0,0,0],[0,0,1,0,0,0,0],[0,0,0,1,0,0,0]])

    self.kf.R[2:,2:] *= 10.
    self.kf.P[4:,4:] *= 1000. #give high uncertainty to the unobservable initial velocities
    self.kf.P *= 10.
    self.kf.Q[-1,-1] *= 0.01
    self.kf.Q[4:,4:] *= 0.01

    self.kf.x[:4] = convert_bbox_to_z(bbox)
    self.time_since_update = 0
    self.id = KalmanBoxTracker.count
    KalmanBoxTracker.count += 1
    self.history = []
    self.hits = 0
    self.hit_streak = 0
    self.age = 0

  def update(self,bbox):
    """
    Updates the state vector with observed bbox.
    """
    self.time_since_update = 0
    self.history = []
    self.hits += 1
    self.hit_streak += 1
    self.kf.update(convert_bbox_to_z(bbox))

  def predict(self):
    """
    Advances the state vector and returns the predicted bounding box estimate.
    """
    if((self.kf.x[6]+self.kf.x[2])<=0):
      self.kf.x[6] *= 0.0
    self.kf.predict()
    self.age += 1
    if(self.time_since_update>0):
      self.hit_streak = 0
    self.time_since_update += 1
    self.history.append(convert_x_to_bbox(self.kf.x))
    return self.history[-1]

  def get_state(self):
    """
    Returns the current bounding box estimate.
    """
    return convert_x_to_bbox(self.kf.x)


def associate_detections_to_trackers(detections,trackers,iou_threshold = 0.3):
  """
  Assigns detections to tracked object (both represented as bounding boxes)

  Returns 3 lists of matches, unmatched_detections and unmatched_trackers
  """
  if(len(trackers)==0):
    return np.empty((0,2),dtype=int), np.arange(len(detections)), np.empty((0,5),dtype=int)

  iou_matrix = iou_batch(detections, trackers)

  if min(iou_matrix.shape) > 0:
    a = (iou_matrix > iou_threshold).astype(np.int32)
    if a.sum(1).max() == 1 and a.sum(0).max() == 1:
        matched_indices = np.stack(np.where(a), axis=1)
    else:
      matched_indices = linear_assignment(-iou_matrix)
  else:
    matched_indices = np.empty(shape=(0,2))

  unmatched_detections = []
  for d, det in enumerate(detections):
    if(d not in matched_indices[:,0]):
      unmatched_detections.append(d)
  unmatched_trackers = []
  for t, trk in enumerate(trackers):
    if(t not in matched_indices[:,1]):
      unmatched_trackers.append(t)

  #filter out matched with low IOU
  matches = []
  for m in matched_indices:
    if(iou_matrix[m[0], m[1]]<iou_threshold):
      unmatched_detections.append(m[0])
      unmatched_trackers.append(m[1])
    else:
      matches.append(m.reshape(1,2))
  if(len(matches)==0):
    matches = np.empty((0,2),dtype=int)
  else:
    matches = np.concatenate(matches,axis=0)

  # print("matches:",matches)
  # print("unmatched detection:",np.array(unmatched_detections))
  # print("unmatched trackers:",np.array(unmatched_trackers))
  return matches, np.array(unmatched_detections), np.array(unmatched_trackers)


class Sort(object):
  def __init__(self, max_age=1, min_hits=1, iou_threshold=0.3):
    """
    Sets key parameters for SORT
    """
    self.max_age = max_age
    self.min_hits = min_hits
    self.iou_threshold = iou_threshold
    self.trackers = []
    self.frame_count = 0
    self.labels = {}  # Dictionary to store labels associated with each tracker

  def update(self, dets=np.empty((0, 6))):
    # self.drop_list.append(1)
    """
    Params:
      dets - a numpy array of detections in the format [[x1,y1,x2,y2,score],[x1,y1,x2,y2,score],...]
    Requires: this method must be called once for each frame even with empty detections (use np.empty((0, 5)) for frames without detections).
    Returns the a similar array, where the last column is the object ID.

    NOTE: The number of objects returned may differ from the number of detections provided.
    """
    self.frame_count += 1
    # get predicted locations from existing trackers.
    trks = np.zeros((len(self.trackers), 6))
    to_del = []
    ret = []
    for t, trk in enumerate(trks):
      pos = self.trackers[t].predict()[0]
      trk[:] = [pos[0], pos[1], pos[2], pos[3], 0, 0]
      if np.any(np.isnan(pos)):
        to_del.append(t)
    trks = np.ma.compress_rows(np.ma.masked_invalid(trks))
    for t in reversed(to_del):
      self.trackers.pop(t)
    matched, unmatched_dets, unmatched_trks = associate_detections_to_trackers(dets,trks, self.iou_threshold)
    # print("len of dets:", len(dets))
    # print("len of matched:", len(matched))
    # print("len of unmatched dets:", len(unmatched_dets))
    # print("len of unmatched trks:", len(unmatched_trks))
    # update matched trackers with assigned detections
    for m in matched:
      trk_id = m[1]
      self.trackers[m[1]].update(dets[m[0], :])

      # Preserve label associated with the tracker---------------------------------------------
      label = dets[m[0], 5]  # Assuming label is in the 6th column
      self.labels[trk_id] = label

    # create and initialise new trackers for unmatched detections
    for i in unmatched_dets:
        trk = KalmanBoxTracker(dets[i,:])
        self.trackers.append(trk)

        # Preserve label for the new tracker-----------------------------------------------------
        label = dets[i, 5]  # Assuming label is in the 6th column
        # self.labels[len(self.trackers) - 1] = label
        self.labels[trk.id] = label


    i = len(self.trackers)
    for trk in reversed(self.trackers):
        trk_id = trk.id
        # label = self.labels[trk_id]
        # d = trk.get_state()[0]
        # if (trk.time_since_update < 1) and (trk.hit_streak >= self.min_hits or self.frame_count <= self.min_hits):
        #   # ret.append(np.concatenate((d,[trk.id+1])).reshape(1,-1)) # +1 as MOT benchmark requires positive
        #   ret.append(np.concatenate((d, [trk.id + 1, label])).reshape(1, -1)) # +1 as MOT benchmark requires positive

        #   # print("ret shape:", ret)

        # # else:
        # #   self.drop_list.pop()
        # #   self.drop_list.append(0)
        # i -= 1
        # # remove dead tracklet
        # if(trk.time_since_update > self.max_age):
        #   self.trackers.pop(i)
        # Check if the trk_id exists in self.labels before accessing it
        if trk_id in self.labels:
            label = self.labels[trk_id]
        else:
            self.labels[trk_id] = label
            # Handle the case when trk_id doesn't exist in self.labels
            label = 99  # Replace "Default Label" with your default label value or handle it according to your logic

        d = trk.get_state()[0]
        if (trk.time_since_update < 1) and (trk.hit_streak >= self.min_hits or self.frame_count <= self.min_hits):
            ret.append(np.concatenate((d, [trk.id + 1, label])).reshape(1, -1))
        i -= 1
        if(trk.time_since_update > self.max_age):
            self.trackers.pop(i)
    if(len(ret)>0):
      return np.concatenate(ret)
    return np.empty((0,6))




In [ ]:
class YOLOv8_ObjectDetector:
    """
    A class for performing object detection on images and videos using YOLOv8.

    Args:
    ------------
        model_file (str): Path to the YOLOv8 model file or yolo model variant name in ths format: [variant].pt
        labels (list[str], optional): A list of class labels for the model. If None, uses the default labels from the model file.
        classes (list[int], optional): The classes we want to detect, use it to exclude certain classes from being detected,
            by default all classes in labels are detectable.
        conf (float, optional): Minimum confidence threshold for object detection.
        iou (float, optional): Minimum IOU threshold for non-max suppression.

    Attributes:
    --------------
        classes (list[str]): A list of class labels for the model ( a Dict is also acceptable).
        conf (float): Minimum confidence threshold for object detection.
        iou (float): Minimum IOU threshold for non-max suppression.
        model (YOLO): The YOLOv8 model used for object detection.
        model_name (str): The name of the YOLOv8 model file (without the .pt extension).

    Methods :
    -------------
        default_display: Returns a default display (ultralytics plot implementation) of the object detection results.
        custom_display: Returns a custom display of the object detection results.
        predict_video: Predicts objects in a video and saves the results to a file.
        predict_img: Predicts objects in an image and returns the detection results.

    """

    def __init__(self, model_file = 'yolov8n.pt', labels= None, classes = None, conf = 0.25, iou = 0.45 ):

        self.classes = classes
        self.conf = conf
        self.iou = iou

        self.model = YOLO(model_file)
        self.model_name = model_file.split('.')[0]
        self.results = None


        # if no labels are provided then use default COCO names
        if labels == None:
            self.labels = self.model.names
        else:
            self.labels = labels

    def predict_img(self, img, verbose=True):
        """
        Runs object detection on a single image.

        Parameters
        ----------
            img (numpy.ndarray): Input image to perform object detection on.
            verbose (bool): Whether to print detection details.

        Returns:
        -----------
            'ultralytics.yolo.engine.results.Results': A YOLO results object that contains
             details about detection results :
                    - Class IDs
                    - Bounding Boxes
                    - Confidence score
                    ...
        (pls refer to https://docs.ultralytics.com/reference/results/#results-api-reference for results API reference)

        """

        # Run the model on the input image with the given parameters
        results = self.model(img, classes=self.classes, conf=self.conf, iou=self.iou, verbose=verbose)

        # Save the original image and the results for further analysis if needed
        self.orig_img = img
        self.results = results[0]

        # Return the detection results
        return results[0]



    def default_display(self, show_conf=True, line_width=None, font_size=None,
                        font='Arial.ttf', pil=False, example='abc'):
        """
        Displays the detected objects on the original input image.

        Parameters
        ----------
        show_conf : bool, optional
            Whether to show the confidence score of each detected object, by default True.
        line_width : int, optional
            The thickness of the bounding box line in pixels, by default None.
        font_size : int, optional
            The font size of the text label for each detected object, by default None.
        font : str, optional
            The font type of the text label for each detected object, by default 'Arial.ttf'.
        pil : bool, optional
            Whether to return a PIL Image object, by default False.
        example : str, optional
            A string to display on the example bounding box, by default 'abc'.

        Returns
        -------
        numpy.ndarray or PIL Image
            The original input image with the detected objects displayed as bounding boxes.
            If `pil=True`, a PIL Image object is returned instead.

        Raises
        ------
        ValueError
            If the input image has not been detected by calling the `predict_img()` method first.
        """
        # Check if the `predict_img()` method has been called before displaying the detected objects
        if self.results is None:
            raise ValueError('No detected objects to display. Call predict_img() method first.')

        # Call the plot() method of the `self.results` object to display the detected objects on the original image
        display_img = self.results.plot(show_conf, line_width, font_size, font, pil, example)

        # Return the displayed image
        return display_img



    def custom_display(self, colors, show_cls = True, show_conf = True):
        """
        Custom display method that draws bounding boxes and labels on the original image,
        with additional options for showing class and confidence information.

        Parameters:
        -----------
        colors : list
            A list of tuples specifying the color of each class.
        show_cls : bool, optional
            Whether to show class information in the label text. Default is True.
        show_conf : bool, optional
            Whether to show confidence information in the label text. Default is True.

        Returns:
        --------
        numpy.ndarray
            The image with bounding boxes and labels drawn on it.
        """

        img = self.orig_img
        # calculate the bounding box thickness based on the image width and height
        bbx_thickness = (img.shape[0] + img.shape[1]) // 450

        for box in self.results.boxes:
            textString = ""

            # Extract object class and confidence score
            score = box.conf.item() * 100
            class_id = int(box.cls.item())

            x1 , y1 , x2, y2 = np.squeeze(box.xyxy.numpy()).astype(int)

            # Print detection info
            if show_cls:
                textString += f"{self.labels[class_id]}"

            if show_conf:
                textString += f" {score:,.2f}%"

            # Calculate font scale based on object size
            font = cv2.FONT_HERSHEY_COMPLEX
            fontScale = (((x2 - x1) / img.shape[0]) + ((y2 - y1) / img.shape[1])) / 2 * 2.5
            fontThickness = 1
            textSize, baseline = cv2.getTextSize(textString, font, fontScale, fontThickness)

            # Draw bounding box, a centroid and label on the image
            img = cv2.rectangle(img, (x1,y1), (x2,y2), colors[class_id], bbx_thickness)
            center_coordinates = ((x1 + x2)//2, (y1 + y2) // 2)

            img =  cv2.circle(img, center_coordinates, 5 , (0,0,255), -1)

             # If there are no details to show on the image
            if textString != "":
                if (y1 < textSize[1]):
                    y1 = y1 + textSize[1]
                else:
                    y1 -= 2
                # show the details text in a filled rectangle
                img = cv2.rectangle(img, (x1, y1), (x1 + textSize[0] , y1 -  textSize[1]), colors[class_id], cv2.FILLED)
                img = cv2.putText(img, textString ,
                    (x1, y1), font,
                    fontScale,  (0, 0, 0), fontThickness, cv2.LINE_AA)

        return img


    def predict_video(self, video_path, save_dir, save_format="avi", display='custom', verbose=True, **display_args):
        """Runs object detection on each frame of a video and saves the output to a new video file.

        Args:
        ----------
            video_path (str): The path to the input video file.
            save_dir (str): The path to the directory where the output video file will be saved.
            save_format (str, optional): The format for the output video file. Defaults to "avi".
            display (str, optional): The type of display for the detection results. Defaults to 'custom'.
            verbose (bool, optional): Whether to print information about the video file and output file. Defaults to True.
            **display_args: Additional arguments to be passed to the display function.

        Returns:
        ------------
            None
        """
        # Open the input video file
        cap = cv2.VideoCapture(video_path)

        # Get the name of the input video file
        vid_name = os.path.basename(video_path)

        # Get the dimensions of each frame in the input video file
        width = int(cap.get(3))  # get `width`
        height = int(cap.get(4))  # get `height`

        # Create the directory for the output video file if it does not already exist
        if not os.path.isdir(save_dir):
            os.makedirs(save_dir)

        # Set the name and path for the output video file
        save_name = self.model_name + ' -- ' + vid_name.split('.')[0] + '.' + save_format
        save_file = os.path.join(save_dir, save_name)

        # Print information about the input and output video files if verbose is True
        if verbose:
            print("----------------------------")
            print(f"DETECTING OBJECTS IN : {vid_name} : ")
            print(f"RESOLUTION : {width}x{height}")
            print('SAVING TO :' + save_file)

        # Define an output VideoWriter object
        out = cv2.VideoWriter(save_file,
                              cv2.VideoWriter_fourcc(*"MJPG"),
                              30, (width, height))

        # Check if the input video file was opened correctly
        if not cap.isOpened():
            print("Error opening video stream or file")

        # Read each frame of the input video file
        while cap.isOpened():
            ret, frame = cap.read()

            # If the frame was not read successfully, break the loop
            if not ret:
                print("Error reading frame")
                break

            # Run object detection on the frame and calculate FPS
            beg = time.time()
            results = self.predict_img(frame, verbose=False)
            if results is None:
                print('***********************************************')
            fps = 1 / (time.time() - beg)

            # Display the detection results
            if display == 'default':
                frame = self.default_display(**display_args)
            elif display == 'custom':
                frame == self.custom_display(**display_args)

            # Display the FPS on the frame
            frame = cv2.putText(frame, f"FPS : {fps:,.2f}",
                                (5, 15), cv2.FONT_HERSHEY_COMPLEX,
                                0.5, (0, 0, 255), 1, cv2.LINE_AA)

            # Write the frame to the output video file
            out.write(frame)

            # Exit the loop if the 'q' button is pressed
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        # After the loop release the cap and video writer
        cap.release()
        out.release()

class YOLOv8_ObjectCounter(YOLOv8_ObjectDetector):
    """
    A class for counting objects in images or videos using the YOLOv8 Object Detection model.

    Attributes:
    -----------
    model_file : str
        The filename of the YOLOv8 object detection model.
    labels : list or None
        The list of labels for the object detection model. If None, the labels will be loaded from the model file.
    classes : list or None
        The list of classes to detect. If None, all classes will be detected.
    conf : float
        The confidence threshold for object detection.
    iou : float
        The Intersection over Union (IoU) threshold for object detection.
    track_max_age : int
        The maximum age (in frames) of a track before it is deleted.
    track_min_hits : int
        The minimum number of hits required for a track to be considered valid.
    track_iou_threshold : float
        The IoU threshold for matching detections to existing tracks.

    Methods:
    --------
    predict_img(img, verbose=True)
        Predicts objects in a single image and counts them.
    predict_video(video_path, save_dir, save_format="avi", display='custom', verbose=True, **display_args)
        Predicts objects in a video and counts them.

    """

    def __init__(self, model_file = 'yolov8n.pt', labels= None, classes = None, conf = 0.25, iou = 0.45,
                 track_max_age = 50, track_min_hits= 10, track_iou_threshold = 0.3 ):

        super().__init__(model_file , labels, classes, conf, iou)

        self.track_max_age = track_max_age
        self.track_min_hits = track_min_hits
        self.track_iou_threshold = track_iou_threshold
        self.class_id_arr = []
        self.result_id_arr = []
        # self.drop_list = []
        self.track_frame_arr = []
        self.distance_arr_x = []
        self.distance_arr_y = []
        self.personCount = []
    def predict_video(self, video_path, save_dir, save_format = "avi",
                      display = 'custom', verbose = True, **display_args):

        """
    Runs object detection on a video file and saves the output as a new video file.

    Args:
        video_path (str): Path to the input video file.
        save_dir (str): Path to the directory where the output video file will be saved.
        save_format (str, optional): Format of the output video file. Defaults to "avi".
        display (str, optional): Type of display to use for object detection results. Options are "default" or "custom".
                                Defaults to "custom".
        verbose (bool, optional): If True, prints information about the input and output video files. Defaults to True.
        **display_args (dict, optional): Additional arguments to pass to the display function.

    Returns:
        None
        """
        cap = cv2.VideoCapture(video_path)
        # Get video name
        vid_name = os.path.basename(video_path)


        # Get frame dimensions and print information about input video file
        width  = int(cap.get(3) )  # get `width`
        height = int(cap.get(4) )  # get `height`

        if not os.path.isdir(save_dir):
            os.makedirs(save_dir)

        save_name = self.model_name + ' -- ' + vid_name.split('.')[0] + '.' + save_format
        save_file = os.path.join(save_dir, save_name)

        if verbose:
            print("----------------------------")
            print(f"DETECTING OBJECTS IN : {vid_name} : ")
            print(f"RESOLUTION : {width}x{height}")
            print('SAVING TO :' + save_file)

        # define an output VideoWriter  object
        out = cv2.VideoWriter(save_file,
                            cv2.VideoWriter_fourcc(*"MJPG"),
                            30,(width,height))

        # Check if the video is opened correctly
        if not cap.isOpened():
            print("Error opening video stream or file")

        # Initialize object tracker
        tracker = Sort(max_age = self.track_max_age, min_hits= self.track_min_hits ,
                            iou_threshold = self.track_iou_threshold)

        # Initialize variables for object counting
        totalCount = []
        personCount =  []
        carCount = []
        currentArray = np.empty((0, 6))
        class_id_arr = []
        result_id_arr = []

        track_frame_arr = np.zeros(1000)
        distance_arr_x = np.zeros(1000)
        distance_arr_y = np.zeros(1000)
        past_value_arr_x = np.zeros(1000)
        past_value_arr_y = np.zeros(1000)
        # Read the video frames
        while cap.isOpened():

            detections = np.empty((0, 6))
            ret, frame = cap.read()

            # If the frame was not read successfully, break the loop
            if not ret:
                print("Error reading frame")
                break

            # Run object detection on the frame and calculate FPS
            beg = time.time()
            results = self.predict_img(frame, verbose = False)
            if results == None:
                print('***********************************************')
            fps = 1 / (time.time() - beg)
            for box in results.boxes:
                score = box.conf.item() * 100
                class_id = int(box.cls.item())#------------------------------------------------------------------------------------------------------------
                # print("class_id: ", class_id)

                # print("class_id_arr: ", class_id_arr)

                x1 , y1 , x2, y2 = np.squeeze(box.xyxy.numpy()).astype(int)

                currentArray = np.array([x1, y1, x2, y2, score, class_id])
                detections = np.vstack((detections, currentArray))

                # # Count the number of each object type
                # if class_id in object_count:
                #     object_count[class_id] += 1
                # else:
                #     object_count[class_id] = 1


            # Update object tracker
            resultsTracker = tracker.update(detections)
            # self.drop_list.append(tracker.drop_list) -----------------------------DROP LIST
            # print(self.drop_list)
            for result in resultsTracker:
                # print("result shape:",result.shape)

                # Get the tracker results
                x1, y1, x2, y2, id, class_id = result
                x1, y1, x2, y2, id, class_id = int(x1), int(y1), int(x2), int(y2), int(id), int(class_id)
                # print("result:",result)----------------------------------------------------------------------
                # result_id_arr.append(id)
                # class_id_arr.append(class_id)
                # print("result_id_arr: ", result_id_arr)
                # print("class_id_arr: ", class_id_arr)

                # if id < 10:
                #   print("result:",result)
                # if id == 1:
                #   id1_arr.append(y1)
                #   print("id1_y1:",id1_arr)
                # if id == 3:
                #   id3_arr.append(y1)
                #   print("id3_y1:",id3_arr)
                # if id == 2:
                #   id2_arr.append(y1)
                #   print("id2_y1:",id2_arr)
                # if id == 4:
                #   id4_arr.append(y1)
                #   print("id4_y1:",id4_arr)
                # if id == 6:
                #   id6_arr.append(y1)
                #   print("id6_y1:",id6_arr)
                # if id == 9:
                #   id9_arr.append(y1)
                #   print("id9_y1:",id9_arr)
                # if id == 24:
                #   id9_arr.append(y1)
                #   print("id24_y1:",id24_arr)

                # Calculate moving distance using y1:


                # Display current objects IDs
                w, h = x2 - x1, y2 - y1
                cx, cy = x1 + w // 2, y1 + h // 2
                id_txt = f"ID: {str(id)}"
                cv2.putText(frame, id_txt, (cx, cy), 4, 0.5, (0, 0, 255), 1)

                # if we haven't seen aprticular object ID before, register it in a list
                if totalCount.count(id) == 0:
                    totalCount.append(id)
                    result_id_arr.append(id)
                    class_id_arr.append(class_id)

                track_frame_arr[id] += 1
                if past_value_arr_y[id] == 0:
                  past_value_arr_y[id] = cy
                else:
                  distance_arr_y[id] += past_value_arr_y[id] - cy

                if past_value_arr_x[id] == 0:
                  past_value_arr_x[id] = cx
                else:
                   distance_arr_x[id] += past_value_arr_x[id] - cx
                # for result_id, class_id in zip(result_id_arr, class_id_arr):
                #   if class_id == 0:
                #     personCount.append(id)

                # for result_id, class_id in zip(result_id_arr, class_id_arr):
                #   if class_id == 2:
                #     carCount.append(result_id)
                    # print("class len:", len(class_id_arr))
                # for result_id, class_id in zip(result_id_arr, class_id_arr):
                if personCount.count(id) == 0 and class_id == 0:
                  personCount.append(id)

                # for result_id, class_id in zip(result_id_arr, class_id_arr):
                if carCount.count(id) == 0 and class_id == 2:
                  carCount.append(id)
            #make sure all elements in class_id_arr and result_id_arr matched
            # n = len(class_id_arr)-len(result_id_arr)
            # if(n!=0):
            #     del class_id_arr[-n:]
            # print("class_id_length: ", len(class_id_arr))
            # print("result_id_length: ", len(result_id_arr))


            # Display detection results
            if display == 'default':
                frame = self.default_display(**display_args)

            elif display == 'custom':
                frame == self.custom_display( **display_args)

            # Display FPS on frame
            frame = cv2.putText(frame,f"FPS : {fps:,.2f}" ,
                                (5,55), cv2.FONT_HERSHEY_COMPLEX,
                            0.5,  (0,255,255), 1, cv2.LINE_AA)

            # Display Counting results
            count_txt = f"TOTAL COUNT : {len(totalCount)}"
            frame = cv2.putText(frame, count_txt, (5,45), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 255), 2)
            # print(totalCount)

            # Display Counting results
            count_txt = f"PERSON COUNT : {len(personCount)}"
            frame = cv2.putText(frame, count_txt, (5,75), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 255), 2)
            # print(personCount)

            # Display Counting results
            count_txt = f"CAR COUNT : {len(carCount)}"
            frame = cv2.putText(frame, count_txt, (5,115), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 255), 2)
            # print(carCount)

            # append frame to the video file
            out.write(frame)

            # the 'q' button is set as the
            # quitting button you may use any
            # desired button of your choice

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        # After the loop release the cap
        cap.release()
        out.release()
        print(len(totalCount))
        print(totalCount)
        # print("mean_id1", np.mean(id1_arr))
        # print("mean_id3", np.mean(id3_arr))
        # print("mean_id2", np.mean(id2_arr))
        # # print("mean_id4", np.mean(id4_arr))
        # print("mean_id6", np.mean(id6_arr))
        # print("mean_id9", np.mean(id9_arr))
        # print("mean_id24", np.mean(id24_arr))
        print("class_id_arr: ", class_id_arr)
        print("result_id_arr: ", result_id_arr)
        # print("class_id_length: ", len(class_id_arr))
        # print("result_id_length: ", len(result_id_arr))
        self.class_id_arr = class_id_arr
        self.result_id_arr = result_id_arr
        self.track_frame_arr = track_frame_arr
        self.distance_arr_x = distance_arr_x
        self.distance_arr_y = distance_arr_y
        self.personCount = personCount

In [ ]:
vid_results_path = '/content/video_object_detection_results'
test_vids_path = '/content/Object-tracking-and-counting-using-YOLOV8/test vids'


if not os.path.isdir(vid_results_path):
    os.makedirs(vid_results_path)

if not os.path.isdir(test_vids_path):
    os.makedirs(test_vids_path)

In [ ]:
# yolo_names = ['yolov8n.pt', 'yolov8m.pt', 'yolov8s.pt',  'yolov8l.pt']
yolo_names = ['yolov8m.pt']
colors = []
for _ in range(80):
    rand_tuple = (random.randint(50, 255), random.randint(50, 255), random.randint(50, 255))
    colors.append(rand_tuple)



In [ ]:
counters = []
for yolo_name in yolo_names:
    counter = YOLOv8_ObjectCounter(yolo_name, conf = 0.60 )
    counters.append(counter)


100%|██████████| 49.7M/49.7M [00:00<00:00, 168MB/s]


In [ ]:
for counter in counters:

    counter.predict_video(video_path= '/content/drive/MyDrive/ECSE415/Colab_Notebooks_Final_Project/st-catherines_drive.mp4', save_dir = vid_results_path, save_format = "avi", display = 'custom', colors = colors)

----------------------------
DETECTING OBJECTS IN : st-catherines_drive.mp4 : 
RESOLUTION : 2562x1440
SAVING TO :/content/video_object_detection_results/yolov8m -- st-catherines_drive.avi
Error reading frame
135
[10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 11, 12, 13, 17, 14, 15, 20, 24, 26, 31, 36, 38, 28, 40, 39, 30, 45, 47, 56, 59, 65, 66, 61, 68, 73, 77, 78, 80, 81, 95, 96, 103, 97, 84, 112, 123, 120, 125, 113, 108, 132, 137, 133, 141, 140, 135, 129, 142, 146, 144, 150, 152, 148, 153, 161, 160, 159, 166, 175, 176, 181, 182, 184, 185, 186, 178, 188, 187, 189, 206, 211, 208, 212, 213, 216, 217, 222, 225, 226, 240, 230, 238, 249, 254, 265, 272, 274, 279, 285, 294, 300, 303, 308, 320, 342, 340, 344, 346, 348, 353, 351, 368, 369, 380, 377, 384, 385, 389, 392, 398, 395, 402, 393, 390, 408, 403, 412, 410, 415, 422, 428, 429, 430, 432, 434]
class_id_arr:  [2, 0, 2, 7, 2, 0, 0, 2, 2, 2, 0, 0, 0, 2, 1, 1, 2, 2, 0, 7, 2, 5, 0, 2, 2, 0, 2, 2, 2, 0, 2, 0, 2, 2, 2, 0, 0, 2, 2, 0, 2, 0, 0, 2, 0, 0, 0, 2, 2, 0

In [ ]:
# St-Catherine
class_id_arr = np.array(counter.class_id_arr)
result_id_arr = np.array(counter.result_id_arr)
distance_arr_x = np.array(counter.distance_arr_x)
distance_arr_y = np.array(counter.distance_arr_y)
track_frame_arr = np.array(counter.track_frame_arr)
# personCount = np.array(counter.personCount)

velocity_arr = np.zeros(1000)
parking_num = 0
moving_num = 0
velocity_index = 0
# velocity_sum = 0
# velocity_non_zero = 0
# velocity_mean = 0
# velocity_non_zero = []
# velovity_median = 0

for i in range (len(distance_arr_x)):
  if (distance_arr_x[i] != 0) and (track_frame_arr[i] != 0) and (distance_arr_y[i] != 0) :
    velocity_arr[i] = np.sqrt(np.power(distance_arr_x[i],2) + np.power(distance_arr_y[i],2))/track_frame_arr[i]

# for j in range (len(velocity_arr)):
#   if velocity_arr[j] > 70:
#     velocity_sum += velocity_arr[j]
#     velocity_non_zero += 1
# velocity_mean = velocity_sum/velocity_non_zero
# for j in range (len(velocity_arr)):
#   if velocity_arr[j] > 70:
#     velocity_non_zero.append(velocity_arr[j])
# velocity_median = np.median(velocity_non_zero, axis = None, out = None)

for index in range (len(result_id_arr)):
  if class_id_arr[index] == 2:
    velocity_index = result_id_arr[index]
    if velocity_arr[velocity_index] < 400 and velocity_arr[velocity_index] > 70:
      parking_num += 1
    else:
      moving_num += 1
# print(velocity_median)
# print("velocity:", velocity_arr)
# print("x: ", distance_arr_x)
# print("y: ",distance_arr_y)
# print("frame: ",track_frame_arr)
print("St-Catherine: Number of parking cars: ", parking_num)
print("St-Catherine:Number of moving cars: ", moving_num)
print("St-Catherine:Number of pedestrians: ", len(counter.personCount))
# print("Number of pedestrians: ", len(personCount))
# print(track_frame_arr)
#print(counter.track_frame_arr)


St-Catherine: Number of parking cars:  54
St-Catherine:Number of moving cars:  12
St-Catherine:Number of pedestrians:  63


In [ ]:
counters2 = []
for yolo_name in yolo_names:
    counter2 = YOLOv8_ObjectCounter(yolo_name, conf = 0.60 )
    counters2.append(counter2)

In [ ]:
for counter2 in counters2:

    counter2.predict_video(video_path= '/content/drive/MyDrive/ECSE415/Colab_Notebooks_Final_Project/mcgill_drive.mp4', save_dir = vid_results_path, save_format = "avi", display = 'custom', colors = colors)

----------------------------
DETECTING OBJECTS IN : mcgill_drive.mp4 : 
RESOLUTION : 2562x1440
SAVING TO :/content/video_object_detection_results/yolov8m -- mcgill_drive.avi
Error reading frame
81
[439, 438, 437, 436, 440, 441, 444, 447, 446, 450, 452, 459, 471, 467, 472, 474, 476, 479, 480, 486, 488, 501, 505, 506, 510, 507, 512, 511, 513, 508, 515, 519, 520, 528, 545, 541, 549, 554, 557, 558, 564, 561, 567, 576, 586, 597, 595, 596, 604, 603, 610, 614, 617, 622, 623, 638, 634, 640, 632, 642, 639, 650, 654, 655, 660, 663, 667, 681, 674, 661, 687, 694, 698, 702, 713, 712, 719, 726, 727, 728, 729]
class_id_arr:  [2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 0, 0, 0, 2, 2, 2, 9, 1, 0, 0, 2, 7, 0, 0, 2, 2, 2, 2, 3, 2, 10, 9, 9, 0, 2, 1, 2, 1, 1, 0, 2, 2, 2, 2, 9, 2, 2, 2, 2, 0, 2, 0, 1, 2, 10, 2, 0, 2, 2, 2, 2, 2, 2, 9, 9, 0, 2, 9, 2, 0, 2, 2, 5, 0, 2, 2, 2, 2, 2]
result_id_arr:  [439, 438, 437, 436, 440, 441, 444, 447, 446, 450, 452, 459, 471, 467, 472, 474, 476, 479, 480, 486, 488, 501, 505, 506,

In [17]:
# McGill
class_id_arr = np.array(counter2.class_id_arr)
result_id_arr = np.array(counter2.result_id_arr)
distance_arr_x = np.array(counter2.distance_arr_x)
distance_arr_y = np.array(counter2.distance_arr_y)
track_frame_arr = np.array(counter2.track_frame_arr)
# personCount = np.array(counter2.personCount)

velocity_arr = np.zeros(1000)
parking_num = 0
moving_num = 0
velocity_index = 0
# velocity_sum = 0
# velocity_non_zero = 0
# velocity_mean = 0
# velocity_non_zero = []
# velovity_median = 0

for i in range (len(distance_arr_x)):
  if (distance_arr_x[i] != 0) and (track_frame_arr[i] != 0) and (distance_arr_y[i] != 0) :
    velocity_arr[i] = np.sqrt(np.power(distance_arr_x[i],2) + np.power(distance_arr_y[i],2))/track_frame_arr[i]

# for j in range (len(velocity_arr)):
#   if velocity_arr[j] > 70:
#     velocity_sum += velocity_arr[j]
#     velocity_non_zero += 1
# velocity_mean = velocity_sum/velocity_non_zero
# for j in range (len(velocity_arr)):
#   if velocity_arr[j] > 70:
#     velocity_non_zero.append(velocity_arr[j])
# velocity_median = np.median(velocity_non_zero, axis = None, out = None)

for index in range (len(result_id_arr)):
  if class_id_arr[index] == 2:
    velocity_index = result_id_arr[index]
    if velocity_arr[velocity_index] < 200 and velocity_arr[velocity_index] > 70:
      parking_num += 1
    else:
      moving_num += 1
# print(velocity_median)
# print("velocity:", velocity_arr)
# print("x: ", distance_arr_x)
# print("y: ",distance_arr_y)
# print("frame: ",track_frame_arr)
print("McGill: Number of parking cars: ", parking_num)
print("McGill:Number of moving cars: ", moving_num)
print("McGill:Number of pedestrians: ", len(counter2.personCount))
# print("Number of pedestrians: ", len(personCount))
# print(track_frame_arr)
#print(counter2.track_frame_arr)

McGill: Number of parking cars:  14
McGill:Number of moving cars:  34
McGill:Number of pedestrians:  16


# **Report**


1.   **Description of the overall approach**

      The objective of this project involves detecting and enumerating objects in two short video sequences while distinguishing between stationary and moving cars within the scenes. The team opted to utilize YOLOv8 as the foundation of our model due to its rapid object detection and classification capabilities per frame. Following detection in each frame, the output undergoes analysis through the Simple Online and Real-time Tracking (SORT) function, verifying the validity of detections for subsequent tracking and assigning tracking IDs. By tallying the unique detections, the team conducts object counting and tracking. To differentiate between stationary and moving cars, an additional function calculates the average velocity of each identified object, enabling a comparison. Higher velocities indicate moving cars, while lower velocities signify stationary cars. The program concludes by displaying the count for each classification.

2.   **Software Package and Routine Used**
* YOLOv8 object detection and classification:

  represents an advanced object detection and classification model known for its speed and accuracy. As an evolution in the YOLO series, this model employs a single neural network to simultaneously predict bounding boxes and class probabilities for objects within an image. YOLOv8 operates by dividing the input image into a grid and generating bounding boxes along with class predictions directly from this grid. Notably, its architecture enables real-time detection of objects within images or video frames.

* SORT:

  The Simple Online and Real-time Tracking (SORT) algorithm is a simple and efficient algorithm that can track multiple objects simultaneously. It relies on object detection bounding boxes and implements a data association technique to match detections with existing tracks based on proximity. SORT utilizes the Kalman filter for prediction and track management, adapting to occlusions and handling object disappearances or reappearances. SORT provides reliable and real-time tracking capabilities and it is suitable under the context of this project.

3. **Summary of Program Output**

* McGill:

  * Ground Truth Value (manually counted):

	Moving car: 27

	Parking car: 15

    Person: 30

 * Program Output Value:

	Moving car: 35

	Parking car: 14

   Person: 16

* St. Catherine:

 * Ground Truth Value (manually counted):

	Moving car: 10

	Parking car: 57

	Person: 73

 * Program Output Value:

	Moving car: 12

	Parking car: 54

    Person: 63

4. **Program Performance and Problems**
* Counting Accuracy:

 * Mcgill:

    moving cars: 70.37%

    parking cars: 93.33%

    pedestrians: 53.33%

 * St-Catherine:

    moving cars: 80.00%

    parking cars: 94.73%

    pedestrians: 86.3%

In comparison to the manually counted values for both videos, we have observed a satisfactory accuracy in detecting parking cars. While this result is promising, our program still faces challenges in accurately determining the status of each car. There are two main reasons for this. Firstly, the detection system tends to identify moving trucks as moving cars due to their similar shapes. This leads to the system detecting additional moving vehicles, causing the count to be higher than what was manually observed. Secondly, both videos are captured using a dash camera. The simultaneous movement of the camera and the observed cars introduces complexities in selecting an appropriate reference velocity. This makes it difficult to accurately determine the status of vehicles by comparing their velocities.
Additionally, there is a considerable loss in the count of pedestrians. From the result output video, it is obvious that certain pedestrians go undetected, resulting in an omission from the overall count. This detection issue is especially noticeable for pedestrians wearing dark clothes or walking under trees. When there isn't enough light on an object, the detection system experiences failures, consequently contributing to a loss in the accurate counting of pedestrians.
It's worth noting that our program performs better overall on videos captured on St-Catherine. This is because St-Catherine has brighter lighting compared to McGill, which enhances the detection of pedestrians. Additionally, St-Catherine is a one-way street, with cars moving in the same direction, making it easier to select a more suitable reference velocity for accurately detecting the status of cars.





In [18]:
from pycocotools.coco import COCO

with open('/content/Object-tracking-and-counting-using-YOLOV8/coco.names') as f:
  cats = f.read().splitlines()

In [19]:
totalcount = np.unique(result_id_arr)
names_arr = []
for id_val in totalcount:
  index_object = np.where(result_id_arr == id_val)[0] # Using np.where to find index
  if len(index_object) > 0: # Check if the ID is found
    index_object = index_object[0] # Select the first index if multiple found
    id_class = class_id_arr[index_object]
    name = cats[id_class]
    names_arr.append(name)

In [21]:
print("total count - Mcgill:",len(names_arr))
name_unique = np.unique(names_arr)
for category in name_unique:
  print(category,":", names_arr.count(category))

total count - Mcgill: 81
bicycle : 5
bus : 1
car : 48
fire hydrant : 2
motorbike : 1
person : 16
traffic light : 7
truck : 1


In [22]:
totalcount = np.unique(counter.result_id_arr)
names_arr = []
for id_val in totalcount:
  index_object = np.where(counter.result_id_arr == id_val)[0] # Using np.where to find index
  if len(index_object) > 0: # Check if the ID is found
    index_object = index_object[0] # Select the first index if multiple found
    id_class = counter.class_id_arr[index_object]
    name = cats[id_class]
    names_arr.append(name)

In [23]:
print("total count - St_Catherine:",len(names_arr))
name_unique = np.unique(names_arr)
for category in name_unique:
  print(category,":", names_arr.count(category))

total count - St_Catherine: 135
bicycle : 5
bus : 1
car : 66
person : 58
pottedplant : 1
traffic light : 2
truck : 2
